# Frames

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time
import matplotlib.pyplot as plt

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement=True
session = InteractiveSession(config=config)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:26:00.0, compute capability: 7.5



# Constants

In [3]:
PATH = "data/"
BATCH_SIZE = 16
EPOCHS = 500
USER_RANGE = range(1, 154)
RANDOM_STATE = 11235
G3 = 29.4
SEQUENCE_LENGTH = 128
NUM_BINS = 10

# Data loading

In [4]:
def read_frame_session(session, normalize):
    def split_to_cycles(rec, lims):
        return [rec[lims[idx]:lims[idx + 1]] for idx in range(0, len(lims) - 2)]

    df = pd.DataFrame(columns=range(3 * SEQUENCE_LENGTH + 1))
    
    i = 0
    for subj in Path('data/raw_data/' + session).glob('subj_*'):
        for rec in subj.glob('*'):
            with open(rec.joinpath('3.txt')) as f:
                # read the content of the file
                lines = list(map(lambda line: [float(x) for x in line.strip().split(',')], f.readlines()))

                # select only relevant portion of the recording and generate cycles
                cycles = range(SEQUENCE_LENGTH, len(lines[0]), SEQUENCE_LENGTH)
                lines = list(map(lambda x: split_to_cycles(x, cycles), lines))

                # select label of measurement
                label = int(subj.name.split('_')[1])

                for cycle in range(len(lines[0])):
                    l = []
                    for line in range(len(lines)):
                        l.extend(lines[line][cycle])
                    l.append(label)
                    df.loc[i] = l
                    i += 1
    
    y = df[df.columns[-1]].values
    y = LabelEncoder().fit_transform(y) + 1
    X = df.drop(df.columns[-1], axis=1).values
    
    if normalize is True:
        X = X / G3
                    
    return X, y

# Handcrafted features

In [5]:
def average_absolute_difference(x, avg_x):
    y = np.abs(x - avg_x)
    return np.mean(y)

In [6]:
def zero_crossing_rate(x):
    counter = 0
    for i in range(1,len(x)):
        if( (x[i-1] <= 0 and x[i] >0) or (x[i-1]>=0 and x[i]<0) ):
            counter = counter +1 
    return counter / len(x)

In [7]:
def feature_extraction(data):
    frames, features = np.shape(data)
    num_features = 19 + 4 * NUM_BINS
    result = np.zeros((frames, num_features))
    
    # Extract 59 features from each row    
    for frame in range(0, frames):       
        ax = data[frame, 0:128]
        ay = data[frame, 129:256]
        az = data[frame, 257:384]
        frame_data = data[frame, :].reshape(128, 3)
        am = np.sqrt(np.sum(np.power(frame_data, 2), axis=1))
        
        min_ax = np.min(ax)
        min_ay = np.min(ay)
        min_az = np.min(az)
        min_am = np.min(am)
        result[frame, 0] = min_ax
        result[frame, 1] = min_ay
        result[frame, 2] = min_az
        result[frame, 3] = min_am

        avg_ax = np.mean(ax)
        avg_ay = np.mean(ay)
        avg_az = np.mean(az)
        avg_am = np.mean(am)
        result[frame, 4] = avg_ax
        result[frame, 5] = avg_ay
        result[frame, 6] = avg_az
        result[frame, 7] = avg_am

        std_ax = np.std(ax)
        std_ay = np.std(ay)
        std_az = np.std(az)
        std_am = np.std(am)
        result[frame, 8]  = std_ax
        result[frame, 9]  = std_ay
        result[frame, 10] = std_az
        result[frame, 11] = std_am

        aad_ax = average_absolute_difference(ax, avg_ax)
        aad_ay = average_absolute_difference(ay, avg_ay)
        aad_az = average_absolute_difference(az, avg_az)
        aad_am = average_absolute_difference(am, avg_am)
        result[frame, 12] = aad_ax
        result[frame, 13] = aad_ay
        result[frame, 14] = aad_az
        result[frame, 15] = aad_am

        zcr_ax = zero_crossing_rate( ax )
        zcr_ay = zero_crossing_rate( ay )
        zcr_az = zero_crossing_rate( az )
        result[frame, 16] = zcr_ax
        result[frame, 17] = zcr_ay
        result[frame, 18] = zcr_az

        histo_ax, bin_edges = np.histogram(ax, NUM_BINS, (-1.5,1.5), density=True)
        histo_ay, bin_edges = np.histogram(ay, NUM_BINS, (-1.5,1.5), density=True)
        histo_az, bin_edges = np.histogram(az, NUM_BINS, (-1.5,1.5), density=True)
        histo_am, bin_edges = np.histogram(am, NUM_BINS, (0,2), density=True)
        for i in range(0, NUM_BINS):
            result[frame, 19 + i] = histo_ax[i]
        for i in range(0, NUM_BINS):
            result[frame, 19 + NUM_BINS+i] = histo_ay[i]
        for i in range(0, NUM_BINS):
            result[frame, 19 + 2 * NUM_BINS + i] = histo_az[i]
        for i in range(0, NUM_BINS):
            result[frame, 19 + 3 * NUM_BINS + i] = histo_am[i]
        
        num_NAN = np.count_nonzero(np.isnan(result[frame, :]))
    
        if (num_NAN != 0):
            result[frame, :] = result[frame-1, :]
        
    return result

# Neural models

In [8]:
def build_resnet(input_shape, nb_classes, file_path):
    n_feature_maps = 64
    input_layer = keras.layers.Input(input_shape)

    # BLOCK 1 
    conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
    conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
    conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum 
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)

    # BLOCK 2 
    conv_x = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=8, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
    conv_y = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
    conv_z = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum 
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)

    # BLOCK 3 
    conv_x = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=8, padding='same')(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
    conv_y = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
    conv_z = keras.layers.Conv1D(filters=n_feature_maps*2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # no need to expand channels because they are equal 
    shortcut_y = keras.layers.BatchNormalization()(output_block_2)
    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation('relu')(output_block_3)

    # FINAL 
    gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)
    output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['categorical_accuracy'])
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss', save_best_only=True)
    callbacks = [reduce_lr,model_checkpoint]

    return callbacks, model

In [9]:
def build_fcn(input_shape, nb_classes, file_path):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)

    conv3 = keras.layers.Conv1D(128, kernel_size=3,padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)

    gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(), 
                  metrics=['categorical_accuracy'])

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, 
                                                  min_lr=0.0001)
    
    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss', 
                                                       save_best_only=True)

    callbacks = [reduce_lr,model_checkpoint]

    return callbacks, model 

# Measurements

## Raw

### Same-day

In [10]:
X, y = read_frame_session("session_1", normalize=False)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [12]:
model = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=11235,
                       verbose=0, warm_start=False)

In [13]:
model.score(X_test, y_test)

0.49272349272349275

### Cross-day

In [14]:
X, y = read_frame_session("session_2", normalize=False)

In [15]:
model.score(X, y)

0.19107016300496102

## Raw normalized

### Same-day

In [16]:
X, y = read_frame_session("session_1", normalize=True)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [18]:
model = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=11235,
                       verbose=0, warm_start=False)

In [19]:
model.score(X_test, y_test)

0.49203049203049204

## Cross-day

In [20]:
X, y = read_frame_session("session_2", normalize=True)

In [21]:
model.score(X, y)

0.19177888022678952

## Handcrafted

### Same-day

In [22]:
X, y = read_frame_session("session_1", normalize=False)

In [23]:
X_feat = feature_extraction(X)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_feat, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [25]:
model = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=11235,
                       verbose=0, warm_start=False)

In [26]:
model.score(X_test, y_test)

0.8738738738738738

### Cross-day

In [27]:
X, y = read_frame_session("session_2", normalize=False)

In [28]:
X_feat = feature_extraction(X)

In [29]:
model.score(X_feat, y)

0.2214032600992204

## Handcrafted normalized

### Same-day

In [30]:
X, y = read_frame_session("session_1", normalize=True)

In [31]:
X_feat = feature_extraction(X)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_feat, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [33]:
model = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=11235,
                       verbose=0, warm_start=False)

In [34]:
model.score(X_test, y_test)

0.8218988218988219

### Cross-day

In [35]:
X, y = read_frame_session("session_2", normalize=True)

In [36]:
X_feat = feature_extraction(X)

In [37]:
model.score(X_feat, y)

0.17519489723600284

## ResNet

### Same-day

In [38]:
X, y = read_frame_session("session_1", normalize=False)

In [39]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [40]:
cb, model = build_resnet((128, 3), 153, "zju_frame_raw_resnet_500.hdf5")

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [42]:
mini_batch_size = int(min(X_train.shape[0]/10, BATCH_SIZE))
start_time = time.time()
hist = model.fit(X_train, y_train, 
                      batch_size=mini_batch_size, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=(X_val, y_val), 
                      callbacks=cb)
duration = time.time() - start_time
y_pred = model.predict(X_test)
keras.backend.clear_session()

Train on 4329 samples, validate on 1443 samples
Epoch 1/500
4329/4329 [==============================] - 8s 2ms/sample - loss: 4.1431 - categorical_accuracy: 0.1275 - val_loss: 9.8917 - val_categorical_accuracy: 0.0076
Epoch 2/500
4329/4329 [==============================] - 4s 832us/sample - loss: 2.7722 - categorical_accuracy: 0.3606 - val_loss: 2.6252 - val_categorical_accuracy: 0.3333
Epoch 3/500
4329/4329 [==============================] - 4s 814us/sample - loss: 1.8298 - categorical_accuracy: 0.6147 - val_loss: 1.8706 - val_categorical_accuracy: 0.5142
Epoch 4/500
4329/4329 [==============================] - 4s 830us/sample - loss: 1.2092 - categorical_accuracy: 0.7648 - val_loss: 1.0409 - val_categorical_accuracy: 0.7956
Epoch 5/500
4329/4329 [==============================] - 4s 837us/sample - loss: 0.8056 - categorical_accuracy: 0.8501 - val_loss: 1.2806 - val_categorical_accuracy: 0.6486
Epoch 6/500
4329/4329 [==============================] - 4s 844us/sample - loss: 0.5833 -

Epoch 48/500
4329/4329 [==============================] - 4s 833us/sample - loss: 0.0352 - categorical_accuracy: 0.9901 - val_loss: 0.2971 - val_categorical_accuracy: 0.9279
Epoch 49/500
4329/4329 [==============================] - 4s 823us/sample - loss: 0.0133 - categorical_accuracy: 0.9975 - val_loss: 0.2160 - val_categorical_accuracy: 0.9508
Epoch 50/500
4329/4329 [==============================] - 4s 827us/sample - loss: 0.0159 - categorical_accuracy: 0.9961 - val_loss: 2.3080 - val_categorical_accuracy: 0.6119
Epoch 51/500
4329/4329 [==============================] - 3s 778us/sample - loss: 0.0398 - categorical_accuracy: 0.9882 - val_loss: 0.5446 - val_categorical_accuracy: 0.8884
Epoch 52/500
4329/4329 [==============================] - 4s 812us/sample - loss: 0.0102 - categorical_accuracy: 0.9979 - val_loss: 0.2349 - val_categorical_accuracy: 0.9515
Epoch 53/500
4329/4329 [==============================] - 4s 831us/sample - loss: 0.0163 - categorical_accuracy: 0.9956 - val_loss

4329/4329 [==============================] - 3s 793us/sample - loss: 0.0051 - categorical_accuracy: 0.9988 - val_loss: 0.2404 - val_categorical_accuracy: 0.9501
Epoch 96/500
4329/4329 [==============================] - 4s 809us/sample - loss: 0.0043 - categorical_accuracy: 0.9988 - val_loss: 0.2014 - val_categorical_accuracy: 0.9605
Epoch 97/500
4329/4329 [==============================] - 4s 834us/sample - loss: 0.0037 - categorical_accuracy: 0.9995 - val_loss: 0.2015 - val_categorical_accuracy: 0.9556
Epoch 98/500
4329/4329 [==============================] - 3s 805us/sample - loss: 0.0170 - categorical_accuracy: 0.9954 - val_loss: 0.5222 - val_categorical_accuracy: 0.9002
Epoch 99/500
4329/4329 [==============================] - 3s 802us/sample - loss: 0.0169 - categorical_accuracy: 0.9963 - val_loss: 0.2188 - val_categorical_accuracy: 0.9508
Epoch 100/500
4329/4329 [==============================] - 3s 803us/sample - loss: 0.0026 - categorical_accuracy: 0.9995 - val_loss: 0.2426 - v

Epoch 142/500
4329/4329 [==============================] - 4s 817us/sample - loss: 0.0016 - categorical_accuracy: 0.9998 - val_loss: 0.2489 - val_categorical_accuracy: 0.9584
Epoch 143/500
4329/4329 [==============================] - 3s 778us/sample - loss: 8.7810e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2227 - val_categorical_accuracy: 0.9612
Epoch 144/500
4329/4329 [==============================] - 3s 803us/sample - loss: 0.0037 - categorical_accuracy: 0.9991 - val_loss: 1.1739 - val_categorical_accuracy: 0.7775
Epoch 145/500
4329/4329 [==============================] - 3s 807us/sample - loss: 0.0047 - categorical_accuracy: 0.9991 - val_loss: 0.2577 - val_categorical_accuracy: 0.9556
Epoch 146/500
4329/4329 [==============================] - 3s 798us/sample - loss: 0.0010 - categorical_accuracy: 1.0000 - val_loss: 0.2455 - val_categorical_accuracy: 0.9591
Epoch 147/500
4329/4329 [==============================] - 3s 800us/sample - loss: 8.8442e-04 - categorical_accuracy: 1.0

4329/4329 [==============================] - 3s 794us/sample - loss: 2.1438e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2319 - val_categorical_accuracy: 0.9640
Epoch 189/500
4329/4329 [==============================] - 3s 800us/sample - loss: 0.0023 - categorical_accuracy: 0.9998 - val_loss: 0.2943 - val_categorical_accuracy: 0.9529
Epoch 190/500
4329/4329 [==============================] - 3s 802us/sample - loss: 0.0086 - categorical_accuracy: 0.9977 - val_loss: 0.2689 - val_categorical_accuracy: 0.9570
Epoch 191/500
4329/4329 [==============================] - 3s 783us/sample - loss: 0.0022 - categorical_accuracy: 0.9995 - val_loss: 0.2505 - val_categorical_accuracy: 0.9550
Epoch 192/500
4329/4329 [==============================] - 3s 805us/sample - loss: 0.0010 - categorical_accuracy: 0.9998 - val_loss: 0.2087 - val_categorical_accuracy: 0.9667
Epoch 193/500
4329/4329 [==============================] - 3s 800us/sample - loss: 0.0015 - categorical_accuracy: 0.9998 - val_loss: 0.

...

Epoch 280/500
4329/4329 [==============================] - 4s 818us/sample - loss: 1.8140e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2218 - val_categorical_accuracy: 0.9667
Epoch 281/500
4329/4329 [==============================] - 3s 808us/sample - loss: 2.1553e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2214 - val_categorical_accuracy: 0.9660
Epoch 282/500
4329/4329 [==============================] - 3s 789us/sample - loss: 2.1650e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2201 - val_categorical_accuracy: 0.9667
Epoch 283/500
4329/4329 [==============================] - 3s 785us/sample - loss: 2.9440e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2220 - val_categorical_accuracy: 0.9667
Epoch 284/500
4329/4329 [==============================] - 4s 821us/sample - loss: 1.6660e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2204 - val_categorical_accuracy: 0.9653
Epoch 285/500
4329/4329 [==============================] - 4s 811us/sample - loss: 1.2916e-05 - categoric

Epoch 326/500
4329/4329 [==============================] - 3s 803us/sample - loss: 1.4114e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2401 - val_categorical_accuracy: 0.9653
Epoch 327/500
4329/4329 [==============================] - 3s 801us/sample - loss: 1.1702e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2362 - val_categorical_accuracy: 0.9647
Epoch 328/500
4329/4329 [==============================] - 3s 807us/sample - loss: 1.3172e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2340 - val_categorical_accuracy: 0.9660
Epoch 329/500
4329/4329 [==============================] - 3s 796us/sample - loss: 2.5745e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2319 - val_categorical_accuracy: 0.9674
Epoch 330/500
4329/4329 [==============================] - 3s 795us/sample - loss: 8.5159e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2310 - val_categorical_accuracy: 0.9681
Epoch 331/500
4329/4329 [==============================] - 3s 794us/sample - loss: 1.0264e-05 - categoric

Epoch 372/500
4329/4329 [==============================] - 3s 801us/sample - loss: 3.8537e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2317 - val_categorical_accuracy: 0.9695
Epoch 373/500
4329/4329 [==============================] - 3s 802us/sample - loss: 1.4321e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2315 - val_categorical_accuracy: 0.9709
Epoch 374/500
4329/4329 [==============================] - 3s 803us/sample - loss: 6.2846e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2326 - val_categorical_accuracy: 0.9709
Epoch 375/500
4329/4329 [==============================] - 3s 802us/sample - loss: 3.6839e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2313 - val_categorical_accuracy: 0.9716
Epoch 376/500
4329/4329 [==============================] - 3s 803us/sample - loss: 4.4922e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2319 - val_categorical_accuracy: 0.9709
Epoch 377/500
4329/4329 [==============================] - 3s 795us/sample - loss: 4.3507e-06 - categoric

Epoch 418/500
4329/4329 [==============================] - 3s 803us/sample - loss: 1.0273e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2363 - val_categorical_accuracy: 0.9695
Epoch 419/500
4329/4329 [==============================] - 3s 805us/sample - loss: 2.8111e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2346 - val_categorical_accuracy: 0.9688
Epoch 420/500
4329/4329 [==============================] - 3s 802us/sample - loss: 3.1266e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2336 - val_categorical_accuracy: 0.9695
Epoch 421/500
4329/4329 [==============================] - 3s 808us/sample - loss: 2.5922e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2336 - val_categorical_accuracy: 0.9695
Epoch 422/500
4329/4329 [==============================] - 3s 800us/sample - loss: 2.3635e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2335 - val_categorical_accuracy: 0.9681
Epoch 423/500
4329/4329 [==============================] - 4s 813us/sample - loss: 5.3561e-06 - categoric

Epoch 464/500
4329/4329 [==============================] - 4s 816us/sample - loss: 3.0827e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2516 - val_categorical_accuracy: 0.9667
Epoch 465/500
4329/4329 [==============================] - 3s 789us/sample - loss: 1.8794e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2501 - val_categorical_accuracy: 0.9674
Epoch 466/500
4329/4329 [==============================] - 3s 794us/sample - loss: 2.2735e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2497 - val_categorical_accuracy: 0.9667
Epoch 467/500
4329/4329 [==============================] - 3s 798us/sample - loss: 2.5940e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2499 - val_categorical_accuracy: 0.9674
Epoch 468/500
4329/4329 [==============================] - 3s 805us/sample - loss: 3.5006e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2491 - val_categorical_accuracy: 0.9667
Epoch 469/500
4329/4329 [==============================] - 3s 800us/sample - loss: 3.5130e-06 - categoric

In [43]:
metrics.accuracy_score(np.argmax(y_test, axis=-1),
                       np.argmax(model.predict(X_test), axis=-1))

0.9688149688149689

### Cross-day

In [44]:
X, y = read_frame_session("session_2", normalize=False)

In [45]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [46]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

In [47]:
metrics.accuracy_score(np.argmax(y, axis=1), y_pred)

0.5408929836995039

## ResNet normalized

### Same-day

In [48]:
X, y = read_frame_session("session_1", normalize=True)

In [49]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [50]:
cb, model = build_resnet((128, 3), 153, "zju_frame_norm_resnet_500.hdf5")

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [52]:
mini_batch_size = int(min(X_train.shape[0]/10, BATCH_SIZE))
start_time = time.time()
hist = model.fit(X_train, y_train, 
                      batch_size=mini_batch_size, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=(X_val, y_val), 
                      callbacks=cb)
duration = time.time() - start_time
y_pred = model.predict(X_test)
keras.backend.clear_session()

Train on 4329 samples, validate on 1443 samples
Epoch 1/500
4329/4329 [==============================] - 6s 1ms/sample - loss: 4.2074 - categorical_accuracy: 0.1116 - val_loss: 9.2915 - val_categorical_accuracy: 0.0069
Epoch 2/500
4329/4329 [==============================] - 4s 818us/sample - loss: 2.8719 - categorical_accuracy: 0.3280 - val_loss: 9.5632 - val_categorical_accuracy: 0.0069
Epoch 3/500
4329/4329 [==============================] - 4s 821us/sample - loss: 2.0493 - categorical_accuracy: 0.5438 - val_loss: 13.0641 - val_categorical_accuracy: 0.0042
Epoch 4/500
4329/4329 [==============================] - 4s 821us/sample - loss: 1.4558 - categorical_accuracy: 0.6881 - val_loss: 19.4571 - val_categorical_accuracy: 0.0173
Epoch 5/500
4329/4329 [==============================] - 4s 816us/sample - loss: 1.0623 - categorical_accuracy: 0.7808 - val_loss: 8.8390 - val_categorical_accuracy: 0.0298
Epoch 6/500
4329/4329 [==============================] - 4s 819us/sample - loss: 0.7535

Epoch 48/500
4329/4329 [==============================] - 3s 785us/sample - loss: 0.0383 - categorical_accuracy: 0.9905 - val_loss: 8.9993 - val_categorical_accuracy: 0.1920
Epoch 49/500
4329/4329 [==============================] - 3s 804us/sample - loss: 0.0134 - categorical_accuracy: 0.9975 - val_loss: 1.0248 - val_categorical_accuracy: 0.7581
Epoch 50/500
4329/4329 [==============================] - 4s 818us/sample - loss: 0.0093 - categorical_accuracy: 0.9984 - val_loss: 1.3918 - val_categorical_accuracy: 0.7013
Epoch 51/500
4329/4329 [==============================] - 3s 797us/sample - loss: 0.0212 - categorical_accuracy: 0.9949 - val_loss: 14.7785 - val_categorical_accuracy: 0.1019
Epoch 52/500
4329/4329 [==============================] - 3s 782us/sample - loss: 0.0450 - categorical_accuracy: 0.9882 - val_loss: 22.7532 - val_categorical_accuracy: 0.0208
Epoch 53/500
4329/4329 [==============================] - 3s 800us/sample - loss: 0.0328 - categorical_accuracy: 0.9915 - val_lo

Epoch 95/500
4329/4329 [==============================] - 3s 778us/sample - loss: 0.0328 - categorical_accuracy: 0.9891 - val_loss: 2.2004 - val_categorical_accuracy: 0.6050
Epoch 96/500
4329/4329 [==============================] - 3s 783us/sample - loss: 0.0166 - categorical_accuracy: 0.9958 - val_loss: 0.5597 - val_categorical_accuracy: 0.8787
Epoch 97/500
4329/4329 [==============================] - 3s 778us/sample - loss: 0.0058 - categorical_accuracy: 0.9993 - val_loss: 0.6574 - val_categorical_accuracy: 0.8683
Epoch 98/500
4329/4329 [==============================] - 3s 794us/sample - loss: 0.0117 - categorical_accuracy: 0.9968 - val_loss: 14.5859 - val_categorical_accuracy: 0.1331
Epoch 99/500
4329/4329 [==============================] - 3s 789us/sample - loss: 0.0161 - categorical_accuracy: 0.9961 - val_loss: 0.2643 - val_categorical_accuracy: 0.9453
Epoch 100/500
4329/4329 [==============================] - 3s 789us/sample - loss: 0.0082 - categorical_accuracy: 0.9975 - val_lo

Epoch 142/500
4329/4329 [==============================] - 3s 788us/sample - loss: 0.0069 - categorical_accuracy: 0.9982 - val_loss: 0.3921 - val_categorical_accuracy: 0.9286
Epoch 143/500
4329/4329 [==============================] - 3s 803us/sample - loss: 0.0011 - categorical_accuracy: 1.0000 - val_loss: 0.2755 - val_categorical_accuracy: 0.9550
Epoch 144/500
4329/4329 [==============================] - 3s 800us/sample - loss: 0.0069 - categorical_accuracy: 0.9979 - val_loss: 2.9398 - val_categorical_accuracy: 0.6064
Epoch 145/500
4329/4329 [==============================] - 3s 794us/sample - loss: 0.0035 - categorical_accuracy: 0.9991 - val_loss: 0.2490 - val_categorical_accuracy: 0.9550
Epoch 146/500
4329/4329 [==============================] - 3s 794us/sample - loss: 0.0012 - categorical_accuracy: 1.0000 - val_loss: 0.3734 - val_categorical_accuracy: 0.9362
Epoch 147/500
4329/4329 [==============================] - 3s 801us/sample - loss: 0.0039 - categorical_accuracy: 0.9995 - va

Epoch 189/500
4329/4329 [==============================] - 3s 798us/sample - loss: 0.0012 - categorical_accuracy: 0.9998 - val_loss: 0.2555 - val_categorical_accuracy: 0.9619
Epoch 190/500
4329/4329 [==============================] - 4s 809us/sample - loss: 9.9624e-04 - categorical_accuracy: 0.9998 - val_loss: 0.2407 - val_categorical_accuracy: 0.9626
Epoch 191/500
4329/4329 [==============================] - 3s 789us/sample - loss: 6.9837e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2443 - val_categorical_accuracy: 0.9612
Epoch 192/500
4329/4329 [==============================] - 3s 794us/sample - loss: 0.0013 - categorical_accuracy: 0.9998 - val_loss: 0.2423 - val_categorical_accuracy: 0.9640
Epoch 193/500
4329/4329 [==============================] - 3s 797us/sample - loss: 3.9291e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2331 - val_categorical_accuracy: 0.9626
Epoch 194/500
4329/4329 [==============================] - 3s 802us/sample - loss: 4.9607e-04 - categorical_accur

...

4329/4329 [==============================] - 3s 791us/sample - loss: 1.0487e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2242 - val_categorical_accuracy: 0.9647
Epoch 282/500
4329/4329 [==============================] - 3s 799us/sample - loss: 4.0717e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2213 - val_categorical_accuracy: 0.9667
Epoch 283/500
4329/4329 [==============================] - 4s 823us/sample - loss: 3.4980e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2222 - val_categorical_accuracy: 0.9647
Epoch 284/500
4329/4329 [==============================] - 4s 831us/sample - loss: 2.7480e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2204 - val_categorical_accuracy: 0.9660
Epoch 285/500
4329/4329 [==============================] - 4s 831us/sample - loss: 8.3537e-05 - categorical_accuracy: 1.0000 - val_loss: 0.3255 - val_categorical_accuracy: 0.9459
Epoch 286/500
4329/4329 [==============================] - 3s 801us/sample - loss: 0.0123 - categorical_accuracy: 0.995

Epoch 327/500
4329/4329 [==============================] - 4s 812us/sample - loss: 1.3739e-04 - categorical_accuracy: 1.0000 - val_loss: 0.2347 - val_categorical_accuracy: 0.9647
Epoch 328/500
4329/4329 [==============================] - 3s 806us/sample - loss: 6.2495e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2317 - val_categorical_accuracy: 0.9647
Epoch 329/500
4329/4329 [==============================] - 3s 805us/sample - loss: 4.4543e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2286 - val_categorical_accuracy: 0.9633
Epoch 330/500
4329/4329 [==============================] - 3s 800us/sample - loss: 3.9969e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2261 - val_categorical_accuracy: 0.9653
Epoch 331/500
4329/4329 [==============================] - 3s 803us/sample - loss: 3.6185e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2253 - val_categorical_accuracy: 0.9647
Epoch 332/500
4329/4329 [==============================] - 3s 808us/sample - loss: 1.2964e-04 - categoric

Epoch 373/500
4329/4329 [==============================] - 3s 791us/sample - loss: 2.2639e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2344 - val_categorical_accuracy: 0.9667
Epoch 374/500
4329/4329 [==============================] - 3s 800us/sample - loss: 1.6687e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2330 - val_categorical_accuracy: 0.9681
Epoch 375/500
4329/4329 [==============================] - 3s 787us/sample - loss: 1.2891e-05 - categorical_accuracy: 1.0000 - val_loss: 0.2306 - val_categorical_accuracy: 0.9674
Epoch 376/500
4329/4329 [==============================] - 3s 790us/sample - loss: 5.9755e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2309 - val_categorical_accuracy: 0.9667
Epoch 377/500
4329/4329 [==============================] - 3s 799us/sample - loss: 6.9981e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2316 - val_categorical_accuracy: 0.9647
Epoch 378/500
4329/4329 [==============================] - 3s 796us/sample - loss: 1.0356e-05 - categoric

Epoch 419/500
4329/4329 [==============================] - 3s 801us/sample - loss: 5.2707e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2423 - val_categorical_accuracy: 0.9695
Epoch 420/500
4329/4329 [==============================] - 4s 822us/sample - loss: 3.5336e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2416 - val_categorical_accuracy: 0.9695
Epoch 421/500
4329/4329 [==============================] - 3s 801us/sample - loss: 8.4750e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2481 - val_categorical_accuracy: 0.9674
Epoch 422/500
4329/4329 [==============================] - 3s 798us/sample - loss: 4.1844e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2474 - val_categorical_accuracy: 0.9681
Epoch 423/500
4329/4329 [==============================] - 3s 782us/sample - loss: 3.5449e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2471 - val_categorical_accuracy: 0.9681
Epoch 424/500
4329/4329 [==============================] - 3s 804us/sample - loss: 2.8946e-06 - categoric

Epoch 465/500
4329/4329 [==============================] - 3s 797us/sample - loss: 1.7019e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2475 - val_categorical_accuracy: 0.9688
Epoch 466/500
4329/4329 [==============================] - 3s 784us/sample - loss: 4.0321e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2518 - val_categorical_accuracy: 0.9674
Epoch 467/500
4329/4329 [==============================] - 3s 788us/sample - loss: 6.9701e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2520 - val_categorical_accuracy: 0.9681
Epoch 468/500
4329/4329 [==============================] - 3s 764us/sample - loss: 1.8681e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2463 - val_categorical_accuracy: 0.9681
Epoch 469/500
4329/4329 [==============================] - 3s 792us/sample - loss: 1.7186e-06 - categorical_accuracy: 1.0000 - val_loss: 0.2450 - val_categorical_accuracy: 0.9681
Epoch 470/500
4329/4329 [==============================] - 3s 802us/sample - loss: 2.4954e-06 - categoric

In [53]:
metrics.accuracy_score(np.argmax(y_test, axis=-1),
                       np.argmax(model.predict(X_test), axis=-1))

0.9695079695079695

### Cross-day

In [54]:
X, y = read_frame_session("session_2", normalize=True)

In [55]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [56]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

In [57]:
metrics.accuracy_score(np.argmax(y, axis=1), y_pred)

0.5332388377037562

## FCN

### Same-day

In [58]:
X, y = read_frame_session("session_1", normalize=False)

In [59]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [60]:
cb, model = build_fcn((128, 3), 153, "zju_frame_raw_fcn_500.hdf5")

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [62]:
mini_batch_size = int(min(X_train.shape[0]/10, BATCH_SIZE))
start_time = time.time()
hist = model.fit(X_train, y_train, 
                      batch_size=mini_batch_size, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=(X_val, y_val), 
                      callbacks=cb)
duration = time.time() - start_time
y_pred = model.predict(X_test)
keras.backend.clear_session()

Train on 4329 samples, validate on 1443 samples
Epoch 1/500
4329/4329 [==============================] - 2s 559us/sample - loss: 4.5860 - categorical_accuracy: 0.0686 - val_loss: 7.8596 - val_categorical_accuracy: 0.0076
Epoch 2/500
4329/4329 [==============================] - 1s 346us/sample - loss: 3.8110 - categorical_accuracy: 0.1860 - val_loss: 3.7306 - val_categorical_accuracy: 0.1337
Epoch 3/500
4329/4329 [==============================] - 1s 340us/sample - loss: 3.1902 - categorical_accuracy: 0.3246 - val_loss: 2.9487 - val_categorical_accuracy: 0.3874
Epoch 4/500
4329/4329 [==============================] - 1s 342us/sample - loss: 2.6241 - categorical_accuracy: 0.4770 - val_loss: 2.6079 - val_categorical_accuracy: 0.3777
Epoch 5/500
4329/4329 [==============================] - 1s 340us/sample - loss: 2.1070 - categorical_accuracy: 0.6142 - val_loss: 1.9881 - val_categorical_accuracy: 0.5939
Epoch 6/500
4329/4329 [==============================] - 1s 341us/sample - loss: 1.7146

Epoch 48/500
4329/4329 [==============================] - 1s 343us/sample - loss: 0.0419 - categorical_accuracy: 0.9931 - val_loss: 0.4873 - val_categorical_accuracy: 0.8690
Epoch 49/500
4329/4329 [==============================] - 1s 337us/sample - loss: 0.0540 - categorical_accuracy: 0.9882 - val_loss: 0.4408 - val_categorical_accuracy: 0.8787
Epoch 50/500
4329/4329 [==============================] - 1s 342us/sample - loss: 0.0421 - categorical_accuracy: 0.9942 - val_loss: 0.4082 - val_categorical_accuracy: 0.8912
Epoch 51/500
4329/4329 [==============================] - 1s 342us/sample - loss: 0.0456 - categorical_accuracy: 0.9915 - val_loss: 0.4121 - val_categorical_accuracy: 0.8905
Epoch 52/500
4329/4329 [==============================] - 1s 344us/sample - loss: 0.0368 - categorical_accuracy: 0.9928 - val_loss: 0.5518 - val_categorical_accuracy: 0.8649
Epoch 53/500
4329/4329 [==============================] - 1s 339us/sample - loss: 0.0594 - categorical_accuracy: 0.9861 - val_loss

4329/4329 [==============================] - 1s 339us/sample - loss: 0.0167 - categorical_accuracy: 0.9977 - val_loss: 0.3090 - val_categorical_accuracy: 0.9245
Epoch 96/500
4329/4329 [==============================] - 1s 340us/sample - loss: 0.0179 - categorical_accuracy: 0.9958 - val_loss: 0.6438 - val_categorical_accuracy: 0.8524
Epoch 97/500
4329/4329 [==============================] - 1s 340us/sample - loss: 0.0092 - categorical_accuracy: 0.9982 - val_loss: 0.3664 - val_categorical_accuracy: 0.9148
Epoch 98/500
4329/4329 [==============================] - 1s 341us/sample - loss: 0.0209 - categorical_accuracy: 0.9958 - val_loss: 0.6452 - val_categorical_accuracy: 0.8441
Epoch 99/500
4329/4329 [==============================] - 1s 337us/sample - loss: 0.0199 - categorical_accuracy: 0.9951 - val_loss: 0.4874 - val_categorical_accuracy: 0.8815
Epoch 100/500
4329/4329 [==============================] - 1s 336us/sample - loss: 0.0214 - categorical_accuracy: 0.9954 - val_loss: 0.2108 - v

Epoch 142/500
4329/4329 [==============================] - 1s 338us/sample - loss: 0.0190 - categorical_accuracy: 0.9951 - val_loss: 0.4443 - val_categorical_accuracy: 0.8995
Epoch 143/500
4329/4329 [==============================] - 1s 338us/sample - loss: 0.0188 - categorical_accuracy: 0.9933 - val_loss: 0.2654 - val_categorical_accuracy: 0.9335
Epoch 144/500
4329/4329 [==============================] - 1s 338us/sample - loss: 0.0068 - categorical_accuracy: 0.9986 - val_loss: 0.3356 - val_categorical_accuracy: 0.9071
Epoch 145/500
4329/4329 [==============================] - 1s 336us/sample - loss: 0.0071 - categorical_accuracy: 0.9988 - val_loss: 0.1957 - val_categorical_accuracy: 0.9515
Epoch 146/500
4329/4329 [==============================] - 1s 337us/sample - loss: 0.0039 - categorical_accuracy: 0.9993 - val_loss: 0.1682 - val_categorical_accuracy: 0.9674
Epoch 147/500
4329/4329 [==============================] - 1s 336us/sample - loss: 0.0031 - categorical_accuracy: 0.9998 - va

Epoch 189/500
4329/4329 [==============================] - 1s 335us/sample - loss: 0.0015 - categorical_accuracy: 1.0000 - val_loss: 0.1708 - val_categorical_accuracy: 0.9647
Epoch 190/500
4329/4329 [==============================] - 1s 334us/sample - loss: 8.6940e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1628 - val_categorical_accuracy: 0.9702
Epoch 191/500
4329/4329 [==============================] - 1s 331us/sample - loss: 0.0019 - categorical_accuracy: 0.9993 - val_loss: 0.1628 - val_categorical_accuracy: 0.9695
Epoch 192/500
4329/4329 [==============================] - 1s 331us/sample - loss: 0.0014 - categorical_accuracy: 0.9998 - val_loss: 0.1777 - val_categorical_accuracy: 0.9667
Epoch 193/500
4329/4329 [==============================] - 1s 335us/sample - loss: 0.0022 - categorical_accuracy: 0.9995 - val_loss: 0.1662 - val_categorical_accuracy: 0.9695
Epoch 194/500
4329/4329 [==============================] - 1s 338us/sample - loss: 7.2986e-04 - categorical_accuracy: 1.0

...

4329/4329 [==============================] - 1s 339us/sample - loss: 0.0099 - categorical_accuracy: 0.9975 - val_loss: 2.5616 - val_categorical_accuracy: 0.6043
Epoch 282/500
4329/4329 [==============================] - 1s 341us/sample - loss: 0.0057 - categorical_accuracy: 0.9986 - val_loss: 0.1633 - val_categorical_accuracy: 0.9626
Epoch 283/500
4329/4329 [==============================] - 1s 331us/sample - loss: 7.0339e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1603 - val_categorical_accuracy: 0.9667
Epoch 284/500
4329/4329 [==============================] - 1s 339us/sample - loss: 4.5297e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1548 - val_categorical_accuracy: 0.9695
Epoch 285/500
4329/4329 [==============================] - 1s 337us/sample - loss: 0.0018 - categorical_accuracy: 0.9998 - val_loss: 0.1673 - val_categorical_accuracy: 0.9688
Epoch 286/500
4329/4329 [==============================] - 1s 333us/sample - loss: 4.7663e-04 - categorical_accuracy: 1.0000 - val_

4329/4329 [==============================] - 1s 338us/sample - loss: 1.1453e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1693 - val_categorical_accuracy: 0.9702
Epoch 328/500
4329/4329 [==============================] - 1s 336us/sample - loss: 3.0841e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1679 - val_categorical_accuracy: 0.9709
Epoch 329/500
4329/4329 [==============================] - 1s 334us/sample - loss: 1.6145e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1732 - val_categorical_accuracy: 0.9695
Epoch 330/500
4329/4329 [==============================] - 1s 335us/sample - loss: 1.8412e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1630 - val_categorical_accuracy: 0.9716
Epoch 331/500
4329/4329 [==============================] - 1s 341us/sample - loss: 1.0864e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1667 - val_categorical_accuracy: 0.9744
Epoch 332/500
4329/4329 [==============================] - 1s 343us/sample - loss: 1.0898e-04 - categorical_accuracy: 1

Epoch 373/500
4329/4329 [==============================] - 1s 335us/sample - loss: 4.2882e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1617 - val_categorical_accuracy: 0.9716
Epoch 374/500
4329/4329 [==============================] - 1s 338us/sample - loss: 4.7154e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1620 - val_categorical_accuracy: 0.9730
Epoch 375/500
4329/4329 [==============================] - 1s 341us/sample - loss: 2.7405e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1604 - val_categorical_accuracy: 0.9737
Epoch 376/500
4329/4329 [==============================] - 1s 339us/sample - loss: 4.5915e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1617 - val_categorical_accuracy: 0.9744
Epoch 377/500
4329/4329 [==============================] - 1s 338us/sample - loss: 2.0171e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1704 - val_categorical_accuracy: 0.9716
Epoch 378/500
4329/4329 [==============================] - 1s 336us/sample - loss: 1.0340e-04 - categoric

Epoch 419/500
4329/4329 [==============================] - 1s 334us/sample - loss: 2.2017e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1789 - val_categorical_accuracy: 0.9681
Epoch 420/500
4329/4329 [==============================] - 1s 337us/sample - loss: 5.8034e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1792 - val_categorical_accuracy: 0.9716
Epoch 421/500
4329/4329 [==============================] - 1s 335us/sample - loss: 6.3416e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1776 - val_categorical_accuracy: 0.9695
Epoch 422/500
4329/4329 [==============================] - 1s 337us/sample - loss: 1.4163e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1788 - val_categorical_accuracy: 0.9709
Epoch 423/500
4329/4329 [==============================] - 1s 333us/sample - loss: 8.4806e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1800 - val_categorical_accuracy: 0.9695
Epoch 424/500
4329/4329 [==============================] - 1s 332us/sample - loss: 1.4059e-04 - categoric

Epoch 465/500
4329/4329 [==============================] - 1s 334us/sample - loss: 5.5169e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1613 - val_categorical_accuracy: 0.9764
Epoch 466/500
4329/4329 [==============================] - 1s 332us/sample - loss: 2.4255e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1631 - val_categorical_accuracy: 0.9757
Epoch 467/500
4329/4329 [==============================] - 1s 341us/sample - loss: 2.7335e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1618 - val_categorical_accuracy: 0.9757
Epoch 468/500
4329/4329 [==============================] - 1s 338us/sample - loss: 8.8865e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1771 - val_categorical_accuracy: 0.9702
Epoch 469/500
4329/4329 [==============================] - 1s 335us/sample - loss: 1.6044e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1715 - val_categorical_accuracy: 0.9709
Epoch 470/500
4329/4329 [==============================] - 1s 340us/sample - loss: 2.9749e-05 - categoric

In [63]:
metrics.accuracy_score(np.argmax(y_test, axis=-1),
                       np.argmax(model.predict(X_test), axis=-1))

0.9681219681219682

### Cross-day

In [64]:
X, y = read_frame_session("session_2", normalize=False)

In [65]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [66]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

In [67]:
metrics.accuracy_score(np.argmax(y, axis=1), y_pred)

0.5759036144578313

## FCN normalized

### Same-day

In [68]:
X, y = read_frame_session("session_1", normalize=True)

In [69]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [70]:
cb, model = build_fcn((128, 3), 153, "zju_frame_norm_fcn_500.hdf5")

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [72]:
mini_batch_size = int(min(X_train.shape[0]/10, BATCH_SIZE))
start_time = time.time()
hist = model.fit(X_train, y_train, 
                      batch_size=mini_batch_size, 
                      epochs=EPOCHS,
                      verbose=True, 
                      validation_data=(X_val, y_val), 
                      callbacks=cb)
duration = time.time() - start_time
y_pred = model.predict(X_test)
keras.backend.clear_session()

Train on 4329 samples, validate on 1443 samples
Epoch 1/500
4329/4329 [==============================] - 2s 537us/sample - loss: 4.5940 - categorical_accuracy: 0.0584 - val_loss: 6.1563 - val_categorical_accuracy: 0.0076
Epoch 2/500
4329/4329 [==============================] - 2s 353us/sample - loss: 3.8162 - categorical_accuracy: 0.1776 - val_loss: 11.6126 - val_categorical_accuracy: 0.0055
Epoch 3/500
4329/4329 [==============================] - 2s 349us/sample - loss: 3.1605 - categorical_accuracy: 0.3356 - val_loss: 9.6713 - val_categorical_accuracy: 0.0111
Epoch 4/500
4329/4329 [==============================] - 2s 363us/sample - loss: 2.5844 - categorical_accuracy: 0.4736 - val_loss: 44.8051 - val_categorical_accuracy: 0.0035
Epoch 5/500
4329/4329 [==============================] - 2s 358us/sample - loss: 2.0765 - categorical_accuracy: 0.6179 - val_loss: 25.2863 - val_categorical_accuracy: 0.0055
Epoch 6/500
4329/4329 [==============================] - 2s 363us/sample - loss: 1.6

4329/4329 [==============================] - 2s 358us/sample - loss: 0.0434 - categorical_accuracy: 0.9915 - val_loss: 37.2970 - val_categorical_accuracy: 0.0423
Epoch 48/500
4329/4329 [==============================] - 2s 353us/sample - loss: 0.0338 - categorical_accuracy: 0.9945 - val_loss: 27.0034 - val_categorical_accuracy: 0.0132
Epoch 49/500
4329/4329 [==============================] - 2s 360us/sample - loss: 0.0365 - categorical_accuracy: 0.9938 - val_loss: 28.1785 - val_categorical_accuracy: 0.0395
Epoch 50/500
4329/4329 [==============================] - 2s 365us/sample - loss: 0.0308 - categorical_accuracy: 0.9949 - val_loss: 14.4803 - val_categorical_accuracy: 0.0956
Epoch 51/500
4329/4329 [==============================] - 2s 357us/sample - loss: 0.0400 - categorical_accuracy: 0.9931 - val_loss: 54.4035 - val_categorical_accuracy: 0.0076
Epoch 52/500
4329/4329 [==============================] - 2s 355us/sample - loss: 0.0305 - categorical_accuracy: 0.9942 - val_loss: 22.709

Epoch 94/500
4329/4329 [==============================] - 2s 349us/sample - loss: 0.0592 - categorical_accuracy: 0.9848 - val_loss: 11.3939 - val_categorical_accuracy: 0.1698
Epoch 95/500
4329/4329 [==============================] - 2s 356us/sample - loss: 0.0131 - categorical_accuracy: 0.9977 - val_loss: 1.5205 - val_categorical_accuracy: 0.6881
Epoch 96/500
4329/4329 [==============================] - 2s 350us/sample - loss: 0.0063 - categorical_accuracy: 0.9986 - val_loss: 0.5768 - val_categorical_accuracy: 0.8600
Epoch 97/500
4329/4329 [==============================] - 2s 352us/sample - loss: 0.0048 - categorical_accuracy: 0.9991 - val_loss: 19.4368 - val_categorical_accuracy: 0.0839
Epoch 98/500
4329/4329 [==============================] - 1s 345us/sample - loss: 0.0035 - categorical_accuracy: 0.9998 - val_loss: 1.9390 - val_categorical_accuracy: 0.6119
Epoch 99/500
4329/4329 [==============================] - 2s 348us/sample - loss: 0.0338 - categorical_accuracy: 0.9915 - val_lo

Epoch 141/500
4329/4329 [==============================] - 2s 348us/sample - loss: 0.0019 - categorical_accuracy: 1.0000 - val_loss: 0.2383 - val_categorical_accuracy: 0.9501
Epoch 142/500
4329/4329 [==============================] - 2s 349us/sample - loss: 0.0019 - categorical_accuracy: 1.0000 - val_loss: 0.6567 - val_categorical_accuracy: 0.8593
Epoch 143/500
4329/4329 [==============================] - 2s 356us/sample - loss: 0.0097 - categorical_accuracy: 0.9988 - val_loss: 28.5699 - val_categorical_accuracy: 0.0575
Epoch 144/500
4329/4329 [==============================] - 2s 355us/sample - loss: 0.0074 - categorical_accuracy: 0.9988 - val_loss: 0.2774 - val_categorical_accuracy: 0.9404
Epoch 145/500
4329/4329 [==============================] - 2s 350us/sample - loss: 0.0314 - categorical_accuracy: 0.9915 - val_loss: 41.4760 - val_categorical_accuracy: 0.0353
Epoch 146/500
4329/4329 [==============================] - 2s 347us/sample - loss: 0.0058 - categorical_accuracy: 0.9991 - 

4329/4329 [==============================] - 1s 343us/sample - loss: 0.0020 - categorical_accuracy: 0.9995 - val_loss: 0.6720 - val_categorical_accuracy: 0.8524
Epoch 188/500
4329/4329 [==============================] - 1s 345us/sample - loss: 0.0107 - categorical_accuracy: 0.9975 - val_loss: 0.9189 - val_categorical_accuracy: 0.8191
Epoch 189/500
4329/4329 [==============================] - 1s 342us/sample - loss: 0.0015 - categorical_accuracy: 1.0000 - val_loss: 0.1800 - val_categorical_accuracy: 0.9584
Epoch 190/500
4329/4329 [==============================] - 1s 343us/sample - loss: 0.0205 - categorical_accuracy: 0.9951 - val_loss: 3.7671 - val_categorical_accuracy: 0.4373
Epoch 191/500
4329/4329 [==============================] - 2s 347us/sample - loss: 0.0223 - categorical_accuracy: 0.9924 - val_loss: 63.2752 - val_categorical_accuracy: 0.0381
Epoch 192/500
4329/4329 [==============================] - 1s 345us/sample - loss: 0.0158 - categorical_accuracy: 0.9945 - val_loss: 3.666

...

4329/4329 [==============================] - 1s 344us/sample - loss: 4.6979e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1780 - val_categorical_accuracy: 0.9647
Epoch 280/500
4329/4329 [==============================] - 1s 342us/sample - loss: 0.0052 - categorical_accuracy: 0.9995 - val_loss: 0.2025 - val_categorical_accuracy: 0.9563
Epoch 281/500
4329/4329 [==============================] - 1s 345us/sample - loss: 8.2869e-04 - categorical_accuracy: 0.9998 - val_loss: 0.1843 - val_categorical_accuracy: 0.9619
Epoch 282/500
4329/4329 [==============================] - 1s 342us/sample - loss: 4.7424e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1844 - val_categorical_accuracy: 0.9626
Epoch 283/500
4329/4329 [==============================] - 1s 341us/sample - loss: 5.1144e-04 - categorical_accuracy: 0.9998 - val_loss: 0.1866 - val_categorical_accuracy: 0.9633
Epoch 284/500
4329/4329 [==============================] - 1s 343us/sample - loss: 0.0020 - categorical_accuracy: 0.9995 - 

Epoch 325/500
4329/4329 [==============================] - 1s 344us/sample - loss: 1.6494e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1759 - val_categorical_accuracy: 0.9633
Epoch 326/500
4329/4329 [==============================] - 1s 346us/sample - loss: 4.5650e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1953 - val_categorical_accuracy: 0.9584
Epoch 327/500
4329/4329 [==============================] - 1s 341us/sample - loss: 1.0425e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1652 - val_categorical_accuracy: 0.9667
Epoch 328/500
4329/4329 [==============================] - 1s 343us/sample - loss: 9.1692e-04 - categorical_accuracy: 0.9995 - val_loss: 0.2081 - val_categorical_accuracy: 0.9543
Epoch 329/500
4329/4329 [==============================] - 1s 342us/sample - loss: 1.5886e-04 - categorical_accuracy: 1.0000 - val_loss: 0.1653 - val_categorical_accuracy: 0.9695
Epoch 330/500
4329/4329 [==============================] - 1s 344us/sample - loss: 9.1968e-05 - categoric

Epoch 371/500
4329/4329 [==============================] - 2s 347us/sample - loss: 3.2087e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1669 - val_categorical_accuracy: 0.9674
Epoch 372/500
4329/4329 [==============================] - 1s 340us/sample - loss: 8.7417e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1634 - val_categorical_accuracy: 0.9688
Epoch 373/500
4329/4329 [==============================] - 1s 343us/sample - loss: 4.0944e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1650 - val_categorical_accuracy: 0.9681
Epoch 374/500
4329/4329 [==============================] - 1s 344us/sample - loss: 3.5975e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1680 - val_categorical_accuracy: 0.9695
Epoch 375/500
4329/4329 [==============================] - 2s 354us/sample - loss: 2.6002e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1642 - val_categorical_accuracy: 0.9716
Epoch 376/500
4329/4329 [==============================] - 1s 341us/sample - loss: 2.3224e-05 - categoric

Epoch 417/500
4329/4329 [==============================] - 1s 344us/sample - loss: 1.3486e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1706 - val_categorical_accuracy: 0.9702
Epoch 418/500
4329/4329 [==============================] - 1s 338us/sample - loss: 3.8159e-04 - categorical_accuracy: 0.9998 - val_loss: 0.1664 - val_categorical_accuracy: 0.9695
Epoch 419/500
4329/4329 [==============================] - 2s 349us/sample - loss: 4.6101e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1717 - val_categorical_accuracy: 0.9681
Epoch 420/500
4329/4329 [==============================] - 1s 344us/sample - loss: 2.9802e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1768 - val_categorical_accuracy: 0.9695
Epoch 421/500
4329/4329 [==============================] - 1s 341us/sample - loss: 3.4133e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1775 - val_categorical_accuracy: 0.9667
Epoch 422/500
4329/4329 [==============================] - 1s 342us/sample - loss: 3.1797e-05 - categoric

Epoch 463/500
4329/4329 [==============================] - 1s 334us/sample - loss: 1.6040e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1772 - val_categorical_accuracy: 0.9674
Epoch 464/500
4329/4329 [==============================] - 1s 343us/sample - loss: 1.0299e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1770 - val_categorical_accuracy: 0.9681
Epoch 465/500
4329/4329 [==============================] - 1s 339us/sample - loss: 1.0605e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1756 - val_categorical_accuracy: 0.9695
Epoch 466/500
4329/4329 [==============================] - 1s 341us/sample - loss: 2.6425e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1768 - val_categorical_accuracy: 0.9695
Epoch 467/500
4329/4329 [==============================] - 1s 340us/sample - loss: 1.5645e-05 - categorical_accuracy: 1.0000 - val_loss: 0.1751 - val_categorical_accuracy: 0.9695
Epoch 468/500
4329/4329 [==============================] - 1s 334us/sample - loss: 1.2892e-05 - categoric

In [73]:
metrics.accuracy_score(np.argmax(y_test, axis=-1),
                       np.argmax(model.predict(X_test), axis=-1))

0.9653499653499653

### Cross-day

In [74]:
X, y = read_frame_session("session_2", normalize=True)

In [75]:
enc = OneHotEncoder()
enc.fit(y.reshape(-1,1))
y = enc.transform(y.reshape(-1, 1)).toarray()
X = X.reshape(-1, 128, 3)

In [76]:
y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=1)

In [77]:
metrics.accuracy_score(np.argmax(y, axis=1), y_pred)

0.5723600283486888